###**Welcome to this tutorial where we will show you how to replicate the results from the [paper](https://arxiv.org/pdf/2312.15398.pdf) titled “Fairness-Aware Structured Pruning in Transformers” in AAAI 2024**###




# Connecting to Colab and unzipping the code file



First, let's connect to Google Drive and navigate to the directory where we'll be working. Once there, make sure to place the [`FASP_github.zip`](https://drive.google.com/file/d/1xQTzzK4-mZ1ny4jRMqtsfY3vaDskVmA5/view?usp=sharing) file. Don't forget to enable the Colab GPU!

In [7]:
from google.colab import drive
# As an example, I am using "cd /content/drive/MyDrive/EMNLP_reproducibility" as my directory
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/PhD/reproducibility/FASP

/content/drive/MyDrive/PhD/reproducibility/FASP


In [ ]:
!unzip -o FASP_github.zip

unzip:  cannot find or open FASP_github.zip, FASP_github.zip.zip or FASP_github.zip.ZIP.


# Creating the script files to launch the experiments

## Computing the performance and bias head scores


In [ ]:
import numpy as np
from os import getgroups
from pathlib import Path

id = 0
folder_name = "experiment_1"
heads_per_sh = 10
for group in ["gender_and_sex"]:
  for model in ["gpt2","distilgpt2","EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","EleutherAI/gpt-j-6B","meta-llama/Llama-2-7b-chat-hf"]:

      if model == "EleutherAI/gpt-neo-1.3B":
        num_all_heads = 384
      elif model == "EleutherAI/gpt-neo-125M":
        num_all_heads = 144
      elif model == "EleutherAI/gpt-j-6B":
        num_all_heads = 448
      elif model == "meta-llama/Llama-2-7b-chat-hf":
        num_all_heads = 1024
      elif model == "gpt2":
        num_all_heads = 144
      elif model == "distilgpt2":
        num_all_heads = 72

      for prompting in ["holistic"]:
        for head_knockout in range(0,num_all_heads,heads_per_sh):
          for seed in range(1,4):
            my_file = open("./scripts/sample.sh")
            string_list = my_file.readlines()

            my_file.close()
            string_list[1] = "#SBATCH --account=def-bengioy_cpu" + "\n"
            string_list[2] = "#SBATCH --cpus-per-task=1" + "\n"
            string_list[3] =  "\n"
            string_list[4] = "#SBATCH --mem=25G" + "\n"
            string_list[8] = "python main.py "

            if model == "EleutherAI/gpt-neo-125M":
              model_name = "N1"
              string_list[4] = "#SBATCH --mem=50G" + "\n"

            elif model == "EleutherAI/gpt-j-6B":
              model_name = "NJ"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=100G" + "\n"
              string_list[8] += " --batch_size 64 "

            elif model == "meta-llama/Llama-2-7b-chat-hf":
              model_name = "L7"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=100G" + "\n"
              string_list[8] += " --batch_size 64 "

            elif model == "EleutherAI/gpt-neo-1.3B":
              model_name = "N2"
              string_list[1] = "#SBATCH --account=def-bengioy_gpu" + "\n"
              string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
              string_list[4] = "#SBATCH --mem=50G" + "\n"
              string_list[8] += " --batch_size 128 "

            elif model == "distilgpt2":
              model_name = "G2D"
            elif model == "gpt2":
              model_name = "G2"
            elif model == "distilroberta-base":
              model_name = "RD"
            elif model == "distilbert-base-cased":
              model_name = "BD"

            string_list[5] = "#SBATCH --time=11:57:00" + "\n"

            if group == "gender_and_sex":
              group_name = "g"
            elif group == "race_ethnicity":
              group_name = "r"
            elif group == "religion":
              group_name = "l"
            elif group == "sexual_orientation":
              group_name = "s"
            elif group == "nationality":
              group_name = "n"

            string_list[8] += " --model " + model

            prompting_abbrev="h"
            string_list[8] += " --targeted_holistic_bias " + group
            string_list[8] += " --prompting " + str(prompting)
            string_list[7] = "sleep " + str(1*id) + "\n"
            file_name = "./scripts/" + folder_name  + "/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_h" + str(head_knockout) + "_"  + str(group_name)
            string_list[8] = string_list[8] + " --seed " + str(seed)
            temp = string_list[8]
            string_list[8] = temp + " --head_knockout " + str(head_knockout) + "\n"
            for h in range(1,heads_per_sh):
              string_list[8] += temp + " --head_knockout " + str(head_knockout + h) + "\n"
            Path("./scripts/" + folder_name).mkdir(parents=True, exist_ok=True)
            my_file = open(file_name + ".sh", "w")
            new_file_contents = "".join(string_list)
            my_file.write(new_file_contents)
            my_file.close()
          id+=1

## Comparing different pruning methods


In [ ]:
import numpy as np
from os import getgroups
from pathlib import Path

id = 0
folder_name = "experiment_2"
for group in ["gender_and_sex","race_ethnicity", "religion", "sexual_orientation", "nationality", "gender_and_sex"]:
  for model in ["gpt2","distilgpt2","EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","EleutherAI/gpt-j-6B","meta-llama/Llama-2-7b-chat-hf"]:
    for pruned_heads_ratio in np.linspace(0,0.2,11,endpoint=True):
      for prompting in ["holistic"]:
        for method in ["FASP", "bias_only", "ppl_only","random_structured", "mask_gradient_l2_structured", "magnitude_l2_structured"]:
          if method == "FASP":
            gammas = ["0.2", "0.3","0.4","0.5","0.6", "0.7"]
          else:
            gammas = ["None"]
          for gamma in gammas:
              for seed in range(1,4):
                my_file = open("./scripts/sample.sh")
                string_list = my_file.readlines()

                my_file.close()

                string_list[1] = "#SBATCH --account=rrg-bengioy-ad_gpu" + "\n"
                string_list[2] = "#SBATCH --cpus-per-task=4" + "\n"
                string_list[3] = "#SBATCH --gres=gpu:1" + "\n"
                string_list[4] = "#SBATCH --mem=100G" + "\n"
                string_list[8] = "python main.py "
                string_list[5] = "#SBATCH --time=2:57:00" + "\n"

                if model == "EleutherAI/gpt-neo-125M":
                  model_name = "N1"
                elif model == "EleutherAI/gpt-neo-1.3B":
                  model_name = "N2"
                  string_list[8] += " --batch_size 32 "
                elif model == "EleutherAI/gpt-j-6B":
                  model_name = "NJ"
                  string_list[5] = "#SBATCH --time=11:57:00" + "\n"
                  string_list[8] += " --batch_size 8 "
                elif model == "meta-llama/Llama-2-7b-chat-hf":
                  model_name = "L7"
                  string_list[5] = "#SBATCH --time=11:57:00" + "\n"
                  string_list[8] += " --batch_size 8 "
                elif model == "distilgpt2":
                  model_name = "G2D"
                elif model == "gpt2":
                  model_name = "G2"

                if group == "gender_and_sex":
                  group_name = "g"
                elif group == "race_ethnicity":
                  group_name = "r"
                elif group == "religion":
                  group_name = "l"
                elif group == "sexual_orientation":
                  group_name = "s"
                elif group == "nationality":
                  group_name = "n"

                string_list[8] += " --model " + model
                string_list[8] += " --method " + str(method)
                string_list[8] += " --pruned_heads_ratio " + str(pruned_heads_ratio)
                if gamma != "None":
                  string_list[8] += " --gamma " + str(gamma)

                if prompting == "holistic":
                  prompting_abbrev="h"
                  string_list[8] += " --targeted_holistic_bias " + group

                if method == "random_structured":
                  method_name = "rn"
                elif method == "FASP":
                  method_name = "f"
                elif method == "magnitude_l2_structured":
                  method_name = "m2"
                elif method == "mask_gradient_l2_structured":
                  method_name = "mg2"
                elif method == "bias_only":
                  method_name = "bo"
                elif method == "ppl_only":
                  method_name = "po"

                string_list[8] += " --prompting " + str(prompting)
                string_list[7] = "sleep " + str(1*id) + "\n"
                file_name = "./scripts/" + folder_name  + "/" + str(prompting_abbrev) + str(seed) + "_" + str(model_name) + "_" + str(method_name) + "_"  + str(pruned_heads_ratio) + "_"  + str(group_name)

                if method == "FASP":
                  file_name += "_" + str(gamma)

                string_list[8] += " --seed " + str(seed) + " \n"
                Path("./scripts/" + folder_name).mkdir(parents=True, exist_ok=True)
                my_file = open(file_name + ".sh", "w")
                new_file_contents = "".join(string_list)
                my_file.write(new_file_contents)
                my_file.close()
              id+=1

# Downloading models

In [ ]:
import torch
from transformers import AutoTokenizer, GPT2Tokenizer, LlamaForCausalLM
from transformers import GPTNeoForCausalLM, GPTJForCausalLM, AutoModelWithLMHead

# This file used to download the models from huggingface and save them in the cached_models folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model_name in ["gpt2","distilgpt2","EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","EleutherAI/gpt-j-6B","meta-llama/Llama-2-7b-chat-hf"]:
    print(model_name)
    if model_name in ["gpt2", "gpt2-medium", "gpt2-large", "distilgpt2",  "gpt2-xl"]:
        model = AutoModelWithLMHead.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B"]:
        model = GPTNeoForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side="left")
        num_heads, num_layers = model.config.num_heads, model.config.num_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["EleutherAI/gpt-j-6B"]:
        model =  GPTJForCausalLM.from_pretrained(model_name,revision="float16", torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left")
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["meta-llama/Llama-2-7b-chat-hf"]:
        print("./saved_models/cached_models/" + model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
        model = LlamaForCausalLM.from_pretrained(model_name, token= 'ENTER_YOUR_TOKEN_HERE').to(device)
        num_heads, num_layers = model.config.num_attention_heads, model.config.num_hidden_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    model.save_pretrained("./saved_models/cached_models/" + model_name)
    tokenizer.save_pretrained("./saved_models/cached_tokenizers/" + model_name)
    print("./saved_models/cached_models/" + model_name)

# Pre-processing
> **Important note: This section is to understand the pre-processing steps, but feel free to skip it if you wish. The pre-processed files `models_config.JSON` and `head_contributions.JSON` are already provided.**

## Creating configuration file for models. This is used to generate the `models_config.JSON`

In [ ]:
from transformers import AutoModelWithLMHead,AutoTokenizer
import torch
from transformers import LlamaForCausalLM, AutoTokenizer, GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoForCausalLM, AutoTokenizer, AutoModelWithLMHead, GPT2Tokenizer, GPTJForCausalLM
import json, torch
head_contributions = {}
model_names = ["gpt2","distilgpt2","EleutherAI/gpt-neo-125M","EleutherAI/gpt-neo-1.3B","EleutherAI/gpt-j-6B","meta-llama/Llama-2-7b-chat-hf"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models_config = {}

for model_name in model_names:

    models_config[model_name] = {}

    if model_name in ["gpt2", "distilgpt2"]:
        model = AutoModelWithLMHead.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")   # Initialize tokenizer
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B"]:
        model = GPTNeoForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.num_heads, model.config.num_layers
        head_dim, max_length = int(model.config.hidden_size/num_heads), model.config.max_position_embeddings

    elif model_name in ["EleutherAI/gpt-j-6B"]:
        model = GPTJForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name, revision="float16",torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        num_heads, num_layers = model.config.n_head, model.config.n_layer
        head_dim, max_length = int(model.config.n_embd/num_heads), model.config.n_positions

    elif model_name in ["meta-llama/Llama-2-7b"]:
        model = LlamaForCausalLM.from_pretrained("./saved_models/cached_models/" + model_name, revision="float16",torch_dtype=torch.float16,).to(device)
        tokenizer = AutoTokenizer.from_pretrained("./saved_models/cached_tokenizers/" + model_name, padding_side="left")
        # number of heads per layer, and number of layers
        num_heads, num_layers = model.params.n_heads, model.params.n_layers
        head_dim, max_length = int(model.params.dim/num_heads), 1024

    #Assume we need 2 sections in the config file, let's call them USERINFO and SERVERCONFIG
    models_config[model_name]["num_heads"] = num_heads
    models_config[model_name]["num_layers"] = num_layers
    models_config[model_name]["head_dim"] = head_dim
    models_config[model_name]["max_length"] = max_length

json.dump(models_config, open("./model/models_config.json", "w"))

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

## Computing the head importance scores. This is used to generate the `head_contributions.JSON`
Similar to magnitude and gradient pruning baselines, FASP gives importance scores to each head. We get these scores as follows: `preprocessing.py --model distilgpt2 gpt2 EleutherAI/gpt-neo-125M EleutherAI/gpt-neo-1.3B EleutherAI/gpt-j-6B  meta-llama/Llama-2-7b-chat-hf`


# Running the code

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

##*As an example, to compute the effect of randomly pruning 18% of the attention heads in DistilGPT2 on gender bias, run the following command:*

In [ ]:
!python main.py  --batch_size 64  --model distilgpt2 --method random_structured --pruned_heads_ratio 0.18 --targeted_holistic_bias gender_and_sex --prompting holistic --seed 3

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
wandb: Tracking run with wandb version 0.16.1
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
/content/drive/MyDrive/PhD/MSR/Text_generation_bias_LLM/our_work/transformers_pruning/models/auto/modeling_auto.py:1062: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (247289 > 1024). Runni

# Collecting the results in csv files

After running the experiments, we gather all the resuts into a single csv file to visualize them, as follows: `python collect_results.py --model  EleutherAI/gpt-neo-125M EleutherAI/gpt-neo-1.3B gpt2 distilgpt2 EleutherAI/gpt-j-6B  meta-llama/Llama-2-7b-chat-hf --experiment compare_to_baselines`.